# かけ算
今回は二進数同士のかけ算を見てみましょう。

## 今回学ぶこと
1. 二進数のかけ算について
2. 回路を作成

## 二進数の掛け算について
２つの数をくらいごとにかけてずらして足し合わせます。その際に量子ビットを利用して桁上がりを考慮します。

0 * 0 = 0  
0 * 1 = 0  
1 * 0 = 0  
1 * 1 = 1  

これはccxゲートの挙動に対応しています。ccxゲートを利用してかけ算を行い、その後各位を足し合わせます。

## 例題
まずは例題を行います。01 * 10 = ?をときます。答えは、01 * 10 = 0010 となります。これを量子回路を利用して解きます。


``` 

   01   a
*  10   b
-------
   00   c
  01    c
-------
  0     z
 0      z
-------
 0010   x
```

では、早速実装へ。まずは2進数の数を２つ用意します。a * bを考えますが、aの0のくらいとaの2の位を用意して、それぞれa0とa2とします。bも同様です。

今回最終的に実現するのは|a,b,x> => |a, b, a * b >とします。求めたいのはx0,x2,x4,x8です。cは途中の計算用のビット。zは桁上がりビットを想定します。

<img src="./img/010_basic_multi01.png">


In [1]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
from blueqat import Circuit

C1 = Circuit().ccx[0,1,2].ccx[1,3,5].ccx[0,4,6].ccx[3,4,7].ccx[5,6,8].ccx[7,8,9].cx[2,10].cx[5,11].cx[6,11].cx[7,12].cx[8,12].cx[9,13] 

#00 * 00 = 0000
(Circuit() + C1).m[:].run(shots=100)

Counter({'00000000000000': 100})

In [17]:
#01 * 01 = 0001
(Circuit().x[0,1] + C1).m[:].run(shots=100)

Counter({'11100000001000': 100})

In [16]:
#10 * 01 = 0010
(Circuit().x[3,1] + C1).m[:].run(shots=100)

Counter({'01010100000100': 100})

In [15]:
#01 * 10 = 0010
(Circuit().x[0,4] + C1).m[:].run(shots=100)

Counter({'10001010000100': 100})

In [14]:
#10 * 10 = 0100
(Circuit().x[3,4] + C1).m[:].run(shots=100) 

Counter({'00011001000010': 100})

In [13]:
#11 * 10 = 0110
(Circuit().x[0,3,4] + C1).m[:].run(shots=100)

Counter({'10011011000110': 100})

In [12]:
#10 * 11 = 0110
(Circuit().x[1,3,4] + C1).m[:].run(shots=100)

Counter({'01011101000110': 100})

In [8]:
#11 * 11 = 1001
(Circuit().x[0,1,3,4] + C1).m[:].run(shots=100) 

Circuit(14).ccx[0, 1, 2].ccx[1, 3, 5].ccx[0, 4, 6].ccx[3, 4, 7].ccx[5, 6, 8].ccx[7, 8, 9].cx[2, 10].cx[5, 11].cx[6, 11].cx[7, 12].cx[8, 12].cx[9, 13]

このように全ての掛け算のパターンが量子回路で実現できました。試しに入力を重ね合わせにしてみます。アダマールゲートをXゲートの代わりに入れると、

In [10]:
(Circuit().h[0,1,3,4] + C1).m[:].run(shots=100) 

Counter({'00010000000000': 6,
         '01011101000110': 8,
         '11110100001100': 8,
         '10011011000110': 8,
         '01000000000000': 4,
         '01001000000000': 8,
         '10010000000000': 4,
         '00000000000000': 9,
         '01010100000100': 5,
         '10001010000100': 6,
         '11100000001000': 7,
         '11111111111001': 5,
         '00011001000010': 6,
         '00001000000000': 11,
         '10000000000000': 3,
         '11101010001100': 2})

上手い感じに 00 * 00 から 11 * 11 までを計算することができました。

# 概要

## 乗算器
ビットは 0, 1 をとるので 二進数のかけ算を考えます。
組み合わせは

0×0=0, 0×1=0, 1×0=0, 1×1=1

となります。
これは加算器の1桁目と同じccxゲートで表せます。

## 乗算器 (2桁×2桁)
次は応用で2桁×2桁の乗算を考えてみます。

組み合わせは

01×00=0000, 01×01=0001, 01×10=0010, 01×11=0011   
11×00=0000, 11×01=0011, 11×10=0110, 11×11=1001   
01×00=0000, 01×01=0001, 01×10=0010, 01×11=0011   
11×00=0000, 11×01=0011, 11×10=0110, 11×11=1001

の16通りとなります。

### 考え方
まず試しに 11 × 10 を筆算で考えてみます。

<img src="./img/010_02/010_02_0.png" width="15%">

1 と 2 の行を合わせた4回の計算をそれぞれ考えます。   
1行目 は 11 × 0 , 2行目は 11 × 1 となります。   
各項4つの計算はそれぞれ 1桁の積でできるので全て CCXで実装できます。

行目1 と 2 の和に関しては繰り上がりを考えないといけないので加算器で実装します。

## 回路の作成（１、２行目）
今回は 1 行目だけ考えます。
11 × 0 を CCX ゲートを用いて以下の回路を考えます。

<img src="./img/010_02/010_02_1.png" width="28%">

上から 2 つのビットは 11 の部分で次の 1 ビットは 0 の部分となります。   
出力_1 ,出力_2 は 1 × 0 を表しています。

これで 1 行目が出力されました。2 行目も同様にしてできます。

## 回路の作成（全体）
11 × 10 を計算します。
上の2つの回路を組み合わせて以下の回路を作ります。

<img src="./img/010_02/010_02_3.png" width="65%">

左上の操作は 1, 2 行目を計算しています。
右下は 1, 2 行目の和を計算しています。